In [ ]:
import numpy as np
from matplotlib import pyplot as plt
# Import the qiskit library
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import plot_histogram

from qiskit import ClassicalRegister
from IPython.display import display
# 追加: fidelity 検証用
from qiskit.quantum_info import DensityMatrix, partial_trace, state_fidelity
# Qiskitライブラリーを導入
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime import QiskitRuntimeService

from qiskit import transpile
from qiskit.transpiler import CouplingMap
from qiskit.providers.fake_provider import GenericBackendV2
n = 23
qcn = QuantumCircuit(n)
qcn.h(0)
qcn.cx(0, 5)
for i in range(1, 5):   # (5,4) (4,3) (3,2) (2,1)
    qcn.cx(6 - i, 5 - i)
for i in range(1, 4):   # (5,6) (6,7) (7,8)
    qcn.cx(4 + i, 5 + i)

# seed = 12
qcn.cx(0, 12)
for i in range(1, 4):   # (12,11) (11,10) (10,9)
    qcn.cx(13 - i, 12 - i)
for i in range(1, 4):   # (12,13) (13,14) (14,15)
    qcn.cx(11 + i, 12 + i)

# seed = 17
qcn.cx(0, 17)
for i in range(1, 2):   # (17,16)
    qcn.cx(18 - i, 17 - i)
for i in range(1, 3):   # (17,18) (18,19)
    qcn.cx(16 + i, 17 + i)

# seed = 20
qcn.cx(0, 20)
for i in range(1, 2):   # (20,21)
    qcn.cx(19 + i, 20 + i)

# ここは N=23 なら有効（0..22 が範囲）
qcn.cx(0, 22)
qcn.draw(output="mpl")
edges = []
for i in range(n):
    j = (i + 1) % n
    edges.append((i, j))
    edges.append((j, i))
edges.append((0, 5))
edges.append((0, 12))
edges.append((0, 17))
edges.append((0, 20))
qcn.measure_all()
coupling_map = CouplingMap(edges)

backend_fake = GenericBackendV2(coupling_map=coupling_map, num_qubits=n)

# 擬似バックエンドでの実行に最適な回路にトランスパイルします
pm = generate_preset_pass_manager(backend=backend_fake, optimization_level=3)
isa_circuit = pm.run(qcn)
isa_circuit.draw("mpl", idle_wires=False)

from qiskit_ibm_runtime import SamplerV2 as Sampler
sampler = Sampler(backend_fake)
job = sampler.run([isa_circuit]) # デフォルトのショット数は1024です

result = job.result()
counts = result[0].data.meas.get_counts()
n = qcn.num_qubits  # 10 のはず
zero_str = "0" * n
one_str = "1" * n
print(f"{zero_str}: {counts.get(zero_str, 0)}")
print(f"{one_str}: {counts.get(one_str, 0)}")
# 必要なら確率
shots = sum(counts.values())
if shots:
    print(f"P({zero_str}) = {counts.get(zero_str,0)/shots:.4f}")
    print(f"P({one_str})  = {counts.get(one_str,0)/shots:.4f}")